## Question Classification Dataset

http://cogcomp.cs.illinois.edu/Data/QA/QC/


In [67]:
%%file load_qc.py
## Question Classification Dataset
## http://cogcomp.cs.illinois.edu/Data/QA/QC/

import numpy as np

def split_question(question):
    q = question.strip().split(" ")
    return (q[0],q[1:])
file

def load_question_file(filename):
    f = open(filename)
    X = list()
    Y = list()
    for line in f:
        (y,x) = split_question(line)
        Y.append(y)
        X.append(x)
    return (Y,X)

def build_dict(sentences):
#    from collections import OrderedDict

    '''
    Build dictionary of train words
    Outputs: 
     - Dictionary of word --> word index
     - Dictionary of word --> word count freq
    '''
    print 'Building dictionary..',
    wordcount = dict()
    #For each worn in each sentence, cummulate frequency
    for ss in sentences:
        for w in ss:
            if w not in wordcount:
                wordcount[w] = 1
            else:
                wordcount[w] += 1

    counts = wordcount.values() # List of frequencies
    keys = wordcount.keys() #List of words
    
    sorted_idx = reversed(np.argsort(counts))
    
    worddict = dict()
    for idx, ss in enumerate(sorted_idx):
        worddict[keys[ss]] = idx+2  # leave 0 and 1 (UNK)
    print np.sum(counts), ' total words ', len(keys), ' unique words'

    return worddict, wordcount

def generate_sequence(sentences, dictionary):
    '''
    Convert tokenized text in sequences of integers
    '''
    seqs = [None] * len(sentences)
    for idx, ss in enumerate(sentences):
        seqs[idx] = [dictionary[w] if w in dictionary else 1 for w in ss]

    return seqs

def parse_label(label):
    t = label.split(":")
    return (t[0],t[1])

def load_corpus(path): 
    (Y_train_full,X_train_sentences) = load_question_file(path + "train_5500.label")
    (Y_test_full,X_test_sentences) = load_question_file(path + "TREC_10.label")
    return (Y_train_full,X_train_sentences), (Y_test_full,X_test_sentences)

def load_data(path): 
    (Y_train_full,X_train_sentences), (Y_test_full,X_test_sentences) = load_corpus(path)
    worddict, wordcount = build_dict(X_train_sentences)
    
    X_train = generate_sequence(X_train_sentences, worddict)
    X_test  = generate_sequence(X_test_sentences, worddict)
    
    Y_train_label = [parse_label(y)[0]  for y in Y_train_full]
    Y_test_label  = [parse_label(y)[0]  for y in Y_test_full]
    
    labels = set(Y_train_label + Y_test_label)
    catdict = {label: idx for (idx, label) in enumerate(labels)}
    
    Y_train = [catdict[y] for y in Y_train_label]
    Y_test  = [catdict[y] for y in Y_test_label]
    
    return (Y_train,X_train), (Y_test,X_test)


Overwriting load_qc.py


In [68]:
path = "/home/ec2-user/data/qc/"
(Y_train,X_train), (Y_test,X_test) = load_data(path)

Building dictionary.. 55635  total words  9448  unique words


In [69]:
nb_classes = len(set(Y_train + Y_test))
print('Num. classes:', nb_classes)
print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')

maxlen_train = max([len(x) for x in X_train])
maxlen_test  = max([len(x) for x in X_test])
print('Max length train', maxlen_train)
print('Max length tet', maxlen_test)

('Num. classes:', 6)
(5452, 'train sequences')
(500, 'test sequences')
('Max length train', 37)
('Max length tet', 17)


In [70]:
print('Pad sequences (samples x time)')
X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

Pad sequences (samples x time)
('X_train shape:', (5452, 30))
('X_test shape:', (500, 30))


In [71]:
from keras.utils import np_utils

print('Convert class vector to binary class matrix (for use with categorical_crossentropy)')
Y_train = np_utils.to_categorical(Y_train, nb_classes)
Y_test = np_utils.to_categorical(Y_test, nb_classes)
print('Y_train shape:', Y_train.shape)
print('Y_test shape:', Y_test.shape)

Convert class vector to binary class matrix (for use with categorical_crossentropy)
('Y_train shape:', (5452, 6))
('Y_test shape:', (500, 6))


## Load models

In [ ]:
# %load models.py

from keras.utils import np_utils

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Embedding
from keras.layers import Convolution1D, MaxPooling1D
from keras.layers import LSTM, GRU, SimpleRNN
from keras.layers import Input, Bidirectional

def build_lstm(max_features, embedding_dims, nb_classes):  
   model = Sequential()

   model.add(Embedding(max_features, 128, dropout=0.2))
   model.add(LSTM(128, dropout_W=0.2, dropout_U=0.2)) 
   model.add(Dense(nb_classes))
   model.add(Activation('softmax'))

   return model


def build_cnn(embedding_dims, maxlen, nb_filter, filter_length, hidden_dims, nb_classes ) : 
    model = Sequential()
    # we start off with an efficient embedding layer which maps
    # our vocab indices into embedding_dims dimensions
    model.add(Embedding(max_features,
                    embedding_dims,
                    input_length=maxlen,
                    dropout=0.2))

    # we add a Convolution1D, which will learn nb_filter
    # word group filters of size filter_length:
    model.add(Convolution1D(nb_filter=nb_filter,
                        filter_length=filter_length,
                        border_mode='valid',
                        activation='relu',
                        subsample_length=1))
    # we use max pooling:
    model.add(MaxPooling1D(pool_length=model.output_shape[1]))

    # We flatten the output of the conv layer,
    # so that we can add a vanilla dense layer:
    model.add(Flatten())

    # We add a vanilla hidden layer:
    model.add(Dense(hidden_dims))
    model.add(Dropout(0.2))
    model.add(Activation('relu'))

    # We project onto a single unit output layer, and squash it with a sigmoid:
    model.add(Dense(nb_classes))
    model.add(Activation('softmax'))
    
    return model


def build_cnn_lstm(embedding_size, maxlen, nb_filter, filter_length, pool_length, lstm_output_size, nb_classes):
    model = Sequential()
    model.add(Embedding(max_features, embedding_size, input_length=maxlen))
    model.add(Dropout(0.25))
    model.add(Convolution1D(nb_filter=nb_filter,
                        filter_length=filter_length,
                        border_mode='valid',
                        activation='relu',
                        subsample_length=1))
    model.add(MaxPooling1D(pool_length=pool_length))
    model.add(LSTM(lstm_output_size))
    model.add(Dense(nb_classes))
    model.add(Activation('softmax'))
     
    return model     

def build_bidirectional_lstm(embedding_dims, lstm_output_size, nb_classes):
    model = Sequential()
    model.add(Embedding(max_features, embedding_dims, input_length=maxlen))
    model.add(Bidirectional(LSTM(lstm_output_size)))
    model.add(Dropout(0.5))
    model.add(Dense(nb_classes, activation='softmax'))
    
    return model

## LSTM 

In [73]:
max_features = 10000
maxlen = 30  # cut texts after this number of words (among top max_features most common words)
batch_size = 32

In [74]:
model = build_lstm(max_features, 128, nb_classes)
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_9 (Embedding)          (None, None, 128)     1280000     embedding_input_9[0][0]          
____________________________________________________________________________________________________
lstm_7 (LSTM)                    (None, 128)           131584      embedding_9[0][0]                
____________________________________________________________________________________________________
dense_10 (Dense)                 (None, 6)             774         lstm_7[0][0]                     
____________________________________________________________________________________________________
activation_9 (Activation)        (None, 6)             0           dense_10[0][0]                   
Total params: 1,412,358
Trainable params: 1,412,358
Non-trainable params: 0
_______________

In [75]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


In [76]:
print('Train...')
model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=15,
          validation_data=(X_test, Y_test))
score, acc = model.evaluate(X_test, Y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

Train...
Train on 5452 samples, validate on 500 samples
Epoch 1/15
5452/5452 [==============================] - 9s - loss: 1.5328 - acc: 0.3347 - val_loss: 1.0397 - val_acc: 0.6360
Epoch 2/15
5452/5452 [==============================] - 9s - loss: 0.8297 - acc: 0.7076 - val_loss: 0.5546 - val_acc: 0.8280
Epoch 3/15
5452/5452 [==============================] - 9s - loss: 0.4297 - acc: 0.8639 - val_loss: 0.4183 - val_acc: 0.8760
Epoch 4/15
5452/5452 [==============================] - 9s - loss: 0.2946 - acc: 0.9109 - val_loss: 0.4139 - val_acc: 0.8980
Epoch 5/15
5452/5452 [==============================] - 9s - loss: 0.2339 - acc: 0.9301 - val_loss: 0.3733 - val_acc: 0.8820
Epoch 6/15
5452/5452 [==============================] - 9s - loss: 0.1726 - acc: 0.9525 - val_loss: 0.4122 - val_acc: 0.8700
Epoch 7/15
5452/5452 [==============================] - 9s - loss: 0.1375 - acc: 0.9622 - val_loss: 0.4204 - val_acc: 0.8760
Epoch 8/15
5452/5452 [==============================] - 9s - loss: 0.

## CNN

In [77]:
max_features = 10000 # vocabulary size
maxlen =  30         # max document length 
batch_size = 32      # minibatch size  

#### Parameters

In [78]:
# set parameters:
embedding_dims = 128 # size of embedding dims
nb_filter = 65      # number of filters 
filter_length = 5    # 1d convolution size
hidden_dims = 250    # size of hidden layers 

In [79]:
cnn_model = build_cnn(embedding_dims, maxlen, nb_filter, filter_length, hidden_dims, nb_classes)
cnn_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_10 (Embedding)         (None, 30, 128)       1280000     embedding_input_10[0][0]         
____________________________________________________________________________________________________
convolution1d_5 (Convolution1D)  (None, 26, 65)        41665       embedding_10[0][0]               
____________________________________________________________________________________________________
maxpooling1d_5 (MaxPooling1D)    (None, 1, 65)         0           convolution1d_5[0][0]            
____________________________________________________________________________________________________
flatten_3 (Flatten)              (None, 65)            0           maxpooling1d_5[0][0]             
___________________________________________________________________________________________

In [80]:
cnn_model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


In [81]:
print('Train...')
cnn_model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=15,
          validation_data=(X_test, Y_test))
score, acc = cnn_model.evaluate(X_test, Y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

Train...
Train on 5452 samples, validate on 500 samples
Epoch 1/15
5452/5452 [==============================] - 1s - loss: 1.2554 - acc: 0.4956 - val_loss: 0.6761 - val_acc: 0.7740
Epoch 2/15
5452/5452 [==============================] - 1s - loss: 0.6236 - acc: 0.7823 - val_loss: 0.4582 - val_acc: 0.8540
Epoch 3/15
5452/5452 [==============================] - 1s - loss: 0.3563 - acc: 0.8819 - val_loss: 0.4013 - val_acc: 0.8680
Epoch 4/15
5452/5452 [==============================] - 1s - loss: 0.2119 - acc: 0.9292 - val_loss: 0.3486 - val_acc: 0.8880
Epoch 5/15
5452/5452 [==============================] - 1s - loss: 0.1650 - acc: 0.9475 - val_loss: 0.3363 - val_acc: 0.8920
Epoch 6/15
5452/5452 [==============================] - 1s - loss: 0.1383 - acc: 0.9569 - val_loss: 0.4174 - val_acc: 0.8740
Epoch 7/15
5452/5452 [==============================] - 1s - loss: 0.1189 - acc: 0.9611 - val_loss: 0.3994 - val_acc: 0.8840
Epoch 8/15
5452/5452 [==============================] - 1s - loss: 0.

## CNN - LSTM

In [82]:
pool_length = 4
lstm_output_size = 70

cnn_lstm_model = build_cnn_lstm(embedding_dims, 
                                maxlen, 
                                nb_filter, 
                                filter_length, 
                                pool_length, 
                                lstm_output_size, 
                                nb_classes)

In [83]:
cnn_lstm_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_11 (Embedding)         (None, 30, 128)       1280000     embedding_input_11[0][0]         
____________________________________________________________________________________________________
dropout_6 (Dropout)              (None, 30, 128)       0           embedding_11[0][0]               
____________________________________________________________________________________________________
convolution1d_6 (Convolution1D)  (None, 26, 65)        41665       dropout_6[0][0]                  
____________________________________________________________________________________________________
maxpooling1d_6 (MaxPooling1D)    (None, 6, 65)         0           convolution1d_6[0][0]            
___________________________________________________________________________________________

In [84]:
cnn_lstm_model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


In [85]:
print('Train...')
cnn_lstm_model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=15,
          validation_data=(X_test, Y_test))
score, acc = cnn_lstm_model.evaluate(X_test, Y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

Train...
Train on 5452 samples, validate on 500 samples
Epoch 1/15
5452/5452 [==============================] - 3s - loss: 1.2916 - acc: 0.4855 - val_loss: 0.6424 - val_acc: 0.7480
Epoch 2/15
5452/5452 [==============================] - 3s - loss: 0.4649 - acc: 0.8485 - val_loss: 0.4631 - val_acc: 0.8560
Epoch 3/15
5452/5452 [==============================] - 3s - loss: 0.1925 - acc: 0.9433 - val_loss: 0.4217 - val_acc: 0.8640
Epoch 4/15
5452/5452 [==============================] - 3s - loss: 0.0842 - acc: 0.9773 - val_loss: 0.4047 - val_acc: 0.8740
Epoch 5/15
5452/5452 [==============================] - 3s - loss: 0.0441 - acc: 0.9881 - val_loss: 0.4854 - val_acc: 0.8540
Epoch 6/15
5452/5452 [==============================] - 3s - loss: 0.0285 - acc: 0.9938 - val_loss: 0.4785 - val_acc: 0.8720
Epoch 7/15
5452/5452 [==============================] - 3s - loss: 0.0181 - acc: 0.9952 - val_loss: 0.5027 - val_acc: 0.8680
Epoch 8/15
5452/5452 [==============================] - 3s - loss: 0.

## Bidirectional LSTM

In [88]:
bilstm_model = build_bidirectional_lstm(embedding_dims, lstm_output_size, nb_classes)
bilstm_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_12 (Embedding)         (None, 30, 128)       1280000     embedding_input_12[0][0]         
____________________________________________________________________________________________________
bidirectional_1 (Bidirectional)  (None, 140)           111440      embedding_12[0][0]               
____________________________________________________________________________________________________
dropout_7 (Dropout)              (None, 140)           0           bidirectional_1[0][0]            
____________________________________________________________________________________________________
dense_14 (Dense)                 (None, 6)             846         dropout_7[0][0]                  
Total params: 1,392,286
Trainable params: 1,392,286
Non-trainable params: 0
_______________

In [89]:
bilstm_model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [90]:
print('Train...')
bilstm_model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=15,
          validation_data=(X_test, Y_test))
score, acc = bilstm_model.evaluate(X_test, Y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

Train...
Train on 5452 samples, validate on 500 samples
Epoch 1/15
5452/5452 [==============================] - 15s - loss: 1.3271 - acc: 0.4582 - val_loss: 0.7474 - val_acc: 0.7500
Epoch 2/15
5452/5452 [==============================] - 15s - loss: 0.4848 - acc: 0.8434 - val_loss: 0.5468 - val_acc: 0.8260
Epoch 3/15
5452/5452 [==============================] - 15s - loss: 0.1718 - acc: 0.9534 - val_loss: 0.4817 - val_acc: 0.8620
Epoch 4/15
5452/5452 [==============================] - 15s - loss: 0.0866 - acc: 0.9776 - val_loss: 0.6549 - val_acc: 0.8360
Epoch 5/15
5452/5452 [==============================] - 15s - loss: 0.0537 - acc: 0.9872 - val_loss: 0.5324 - val_acc: 0.8740
Epoch 6/15
5452/5452 [==============================] - 15s - loss: 0.0317 - acc: 0.9923 - val_loss: 0.6299 - val_acc: 0.8520
Epoch 7/15
5452/5452 [==============================] - 15s - loss: 0.0181 - acc: 0.9961 - val_loss: 0.7442 - val_acc: 0.7540
Epoch 8/15
5452/5452 [==============================] - 15s - 